# 1.1.3.2: Determinante de una Matriz

## Objetivos de Aprendizaje

Al completar este notebook, serás capaz de:

- **Calcular** el determinante de matrices 2x2 y 3x3 usando NumPy.
- **Interpretar** el valor absoluto del determinante como el **factor de escala del área/volumen** de una transformación lineal.
- **Interpretar** el signo del determinante en términos de la **orientación** del espacio (si la transformación "voltea" el espacio).
- **Utilizar** el determinante como una prueba definitiva para determinar si una matriz es **invertible**.
- **Aplicar** el determinante a una matriz de covarianza para obtener una medida de la **dispersión multivariada de los datos**.

In [ ]:
# --- Celda de Configuración (Oculta) ---
%display latex
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

def plot_transformation_area(matrix, ax=None, title='Transformación del Área Unitaria'):
    standalone = ax is None
    if standalone:
        fig, ax = plt.subplots(figsize=(7, 7))
    
    # Vectores base y cuadrado unitario
    i_hat = np.array([1, 0]); j_hat = np.array([0, 1])
    unit_square = Polygon([(0,0), i_hat, i_hat + j_hat, j_hat], facecolor='lightblue', alpha=0.7, label='Área Original = 1')
    ax.add_patch(unit_square)

    # Vectores transformados y paralelogramo
    i_hat_T = matrix @ i_hat; j_hat_T = matrix @ j_hat
    transformed_parallelogram = Polygon([(0,0), i_hat_T, i_hat_T + j_hat_T, j_hat_T], 
                                          facecolor='lightcoral', alpha=0.7, label=f'Nueva Área = |{np.linalg.det(matrix):.2f}|')
    ax.add_patch(transformed_parallelogram)

    # Flechas de los vectores transformados
    ax.quiver(0, 0, i_hat_T[0], i_hat_T[1], angles='xy', scale_units='xy', scale=1, color='#0072B2', zorder=2, label='T(î)')
    ax.quiver(0, 0, j_hat_T[0], j_hat_T[1], angles='xy', scale_units='xy', scale=1, color='#E69F00', zorder=2, label='T(ĵ)')
    
    # Estilo del gráfico
    all_points = np.array([[0,0], i_hat_T, j_hat_T, i_hat_T+j_hat_T])
    limit = np.max(np.abs(all_points)) * 1.2 + 1
    ax.set_xlim(-limit, limit); ax.set_ylim(-limit, limit)
    ax.set_aspect('equal'); ax.grid(True, linestyle='--'); ax.legend()
    ax.set_title(title)
    ax.axhline(0, color='black', lw=0.5); ax.axvline(0, color='black', lw=0.5)
    
    if standalone:
        plt.show()

--- 
## ⚙️ El Arsenal de Datasets: Nuestra Fuente de Ejercicios

El determinante es una propiedad de las matrices cuadradas que describe su comportamiento como transformaciones. Por ello, usaremos nuestro generador de matrices especiales para crear transformaciones con determinantes interesantes (positivos, negativos, cero, uno) y nuestro generador de datos para explorar cómo el determinante de una matriz de covarianza mide la dispersión de los datos.

In [ ]:
# === CONFIGURACIÓN DE DATASETS ===
from src.data_generation.create_student_performance import create_student_performance_data
from src.data_generation.create_special_matrices import create_special_matrices
from src.data_generation.create_edge_cases import create_edge_cases

# Configuración centralizada de aleatoriedad para REPRODUCIBILIDAD
rng = np.random.default_rng(seed=42)

# === Generación de Datasets y Matrices para este Notebook ===

# 💡 CONTEXTO PEDAGÓGICO: Hilo Conductor (Datos para medir dispersión)
# Usaremos el dataset de estudiantes para un concepto avanzado: el determinante de la matriz
# de covarianza como una medida del 'volumen' o 'área' que ocupa la nube de datos.
datos_estudiantes = create_student_performance_data(rng, simplified=True, n_samples=200)

# 💡 CONTEXTO PEDAGÓGICO: Matrices para Visualización Geométrica
# Generaremos matrices con determinantes específicos para visualizar sus efectos.
matriz_escala = np.array([[2, 0.5], [0.5, 2]])  # det > 1 (expande)
matriz_reflexion = np.array([[-1, 0], [0, 1]]) # det < 0 (invierte orientación)
matriz_singular = create_special_matrices(rng, matrix_type='singular', size=(2, 2)) # det = 0 (colapsa)
matriz_rotacion = create_special_matrices(rng, matrix_type='orthogonal', size=(2,2)) # det = 1 (preserva área)
    
print("Datasets y matrices generados y listos para usar.")

## 1. ¿Qué es el Determinante?

El determinante es un valor escalar especial que solo se puede calcular para **matrices cuadradas**. Se denota como `det(A)` o `|A|`.

Para una matriz 2x2: $A = \begin{pmatrix} a & b \\ c & d \end{pmatrix}$, el determinante es la fórmula que probablemente recuerdes:
$$ \det(A) = ad - bc $$

Para matrices más grandes, el cálculo se vuelve más complejo, pero la intuición geométrica es la misma. Afortunadamente, nunca lo calculamos a mano; usamos `np.linalg.det()`.

### La Gran Intuición Geométrica: Factor de Escala de Área/Volumen

El determinante es la respuesta a la pregunta: **¿Por cuánto se multiplica el área (en 2D) o el volumen (en 3D) después de aplicar la transformación lineal representada por la matriz A?**

$$ \text{Área}_{\text{nueva}} = |\det(A)| \times \text{Área}_{\text{original}} $$

- **Valor Absoluto, $|\det(A)|$**: Es el **factor de escala**. Si el determinante es 2, el área se duplica. Si es 0.5, el área se reduce a la mitad.
- **Signo de $\det(A)$**: Indica si la transformación **invierte la orientación** del espacio. 
  - **$\det(A) > 0$**: Preserva la orientación. Imagina que el vector base $\hat{j}$ está a 90° en sentido antihorario de $\hat{i}$. Después de la transformación, $T(\hat{j})$ seguirá estando en la misma dirección relativa a $T(\hat{i})$. Una rotación es un ejemplo clásico.
  - **$\det(A) < 0$**: Invierte la orientación. Es como si el espacio se "volteara" como un calcetín. Una reflexión (como un espejo) es el ejemplo perfecto.

### Ejemplo Demostrativo 1: Visualización del Factor de Escala y Orientación

In [ ]:
# 1. DATOS: Usamos las matrices especiales que generamos.

# 2. VISUALIZACIÓN
fig, axs = plt.subplots(1, 3, figsize=(21, 7))

plot_transformation_area(matriz_escala, ax=axs[0], 
                         title=f'Expansión de Área: det={np.linalg.det(matriz_escala):.2f}')

plot_transformation_area(matriz_reflexion, ax=axs[1], 
                         title=f'Inversión de Orientación: det={np.linalg.det(matriz_reflexion):.2f}')

plot_transformation_area(matriz_rotacion, ax=axs[2], 
                         title=f'Preservación de Área: det={np.linalg.det(matriz_rotacion):.2f}')

plt.tight_layout()
plt.show()

print("Observa cómo T(ĵ) está a la derecha de T(î) en el gráfico del medio, indicando una inversión de la orientación original.")

## 2. Determinantes e Invertibilidad

> **Regla de Oro:** Una matriz cuadrada $A$ es **invertible** (existe $A^{-1}$) si y solo si $\det(A) \neq 0$.

**La Conexión Geométrica:** Si $\det(A) = 0$, significa que la transformación ha "aplastado" el espacio a una dimensión inferior (e.g., un plano 2D se colapsa en una línea o un punto). Esta acción es irreversible, ya que múltiples vectores de entrada diferentes pueden aterrizar en el mismo punto de salida. No hay forma de saber de cuál de ellos provino originalmente, por lo que no se puede "deshacer" la transformación. La información se ha perdido.

### Ejemplo Demostrativo 2: Colapso del Espacio (Determinante Cero)

In [ ]:
# 1. DATOS: Nuestra matriz singular generada, que tiene columnas dependientes.

# 2. APLICACIÓN Y VISUALIZACIÓN
det_singular = np.linalg.det(matriz_singular)
print(f"La matriz singular es:\n{np.round(matriz_singular, 2)}")
print(f"Su determinante es (aproximadamente): {det_singular:.10f}")
print("Un determinante tan cercano a cero indica que la matriz no es invertible.")

plot_transformation_area(matriz_singular, title='Colapso del Espacio: det=0')

### Ejemplo Demostrativo 3: Determinante como Medida de Dispersión de Datos

In [ ]:
# 1. DATOS: Usamos el Hilo Conductor y creamos una versión altamente correlacionada.
datos_normales = datos_estudiantes[['horas_estudio', 'calificacion_examen']].values
datos_correlacionados = create_edge_cases(rng, case_type='multicollinear', n_samples=200)[['x1', 'x2']].values

# 2. APLICACIÓN: Calculamos la matriz de covarianza y su determinante para ambos.
cov_normal = np.cov(datos_normales, rowvar=False)
det_normal = np.linalg.det(cov_normal)

cov_corr = np.cov(datos_correlacionados, rowvar=False)
det_corr = np.linalg.det(cov_corr)

# 3. INTERPRETACIÓN
print(f"Determinante de Cov(Datos Normales): {det_normal:.2f}")
print(f"Determinante de Cov(Datos Correlacionados): {det_corr:.2f}")
print("\nUn determinante mucho más pequeño indica que los datos están menos dispersos y más alineados en una dirección.")

# 4. VISUALIZACIÓN
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
ax1.scatter(datos_normales[:, 0], datos_normales[:, 1], alpha=0.5)
ax1.set_title(f'Datos Normales (Área de Dispersión ~ {det_normal:.2f})')
ax1.set_aspect('equal')
ax2.scatter(datos_correlacionados[:, 0], datos_correlacionados[:, 1], alpha=0.5, c='orange')
ax2.set_title(f'Datos Correlacionados (Área de Dispersión ~ {det_corr:.2f})')
ax2.set_aspect('equal')
plt.show()

---
## 4. Ejercicios Guiados con Scaffolding (8+)
Rellena las partes marcadas con `# COMPLETAR` para afianzar tu comprensión.

### === EJERCICIO GUIADO 1: Cálculo e Invertibilidad ===

In [ ]:
# DATOS
A = np.array([[6, -2], [3, 1]])

# TODO 1: Calcula el determinante de A.
# PISTA: Usa np.linalg.det().
det_A = # COMPLETAR

# TODO 2: Escribe una expresión booleana para determinar si es invertible.
es_invertible = # COMPLETAR (debe ser True o False)

# VERIFICACIÓN
assert np.isclose(det_A, 12.0)
assert es_invertible
print(f"✅ ¡Correcto! El determinante es {det_A}, que es no nulo, por lo tanto la matriz es invertible.")

### === EJERCICIO GUIADO 2: Encontrar un Valor que Haga una Matriz Singular ===

In [ ]:
# DATOS
# C = [[4, 6], [2, k]]

# Queremos encontrar el valor de 'k' que hace que det(C) = 0.
# La fórmula es: 4*k - 6*2 = 0  =>  4k = 12

# TODO: Resuelve para k.
k = # COMPLETAR

# VERIFICACIÓN
C = np.array([[4, 6], [2, k]])
det_C = np.linalg.det(C)
assert np.isclose(det_C, 0)
print(f"✅ ¡Correcto! Con k={k}, el determinante es {det_C:.2f} y la matriz es singular (no invertible).")
plot_transformation_area(C)

### === EJERCICIO GUIADO 3: Interpretación Geométrica ===

In [ ]:
# DATOS: Una matriz de reflexión y escalado.
T = np.array([[0, -2], [1, 0]])

# TODO 1: Calcula el determinante.
det_T = # COMPLETAR

# TODO 2: ¿Cuánto cambia el área? (el factor de escala).
factor_escala = # COMPLETAR (Pista: valor absoluto)

# TODO 3: ¿Se invierte la orientación? (True/False)
orientacion_invertida = # COMPLETAR (Pista: signo)

# VERIFICACIÓN
assert np.isclose(det_T, 2.0)
assert factor_escala == 2.0
assert not orientacion_invertida
print("✅ ¡Interpretación correcta!")
print(f"El área se multiplica por {factor_escala} y la orientación se preserva.")
plot_transformation_area(T)

### === EJERCICIO GUIADO 4: Determinante de una Matriz 3x3 ===

In [ ]:
# DATOS
M = create_special_matrices(rng, 'random', (3, 3))

# TODO: Calcula el determinante de la matriz 3x3 M.
det_M = # COMPLETAR

print(f"Matriz M:\n{np.round(M, 2)}")
print(f"\n✅ Determinante calculado: {det_M:.3f}")
print("Interpretación: Esta transformación escala el VOLUMEN por un factor de |det(M)|.")

### === EJERCICIO GUIADO 5: Propiedad det(A.T) = det(A) ===

In [ ]:
# DATOS
A = create_special_matrices(rng, 'random', (4, 4))

# TODO 1: Calcula el determinante de A.
det_A = # COMPLETAR

# TODO 2: Calcula el determinante de la transpuesta de A, A.T.
det_AT = # COMPLETAR

# VERIFICACIÓN
assert np.isclose(det_A, det_AT), "Los determinantes de A y A.T deberían ser iguales."
print("✅ ¡Propiedad verificada!")
print(f"det(A) = {det_A:.4f}")
print(f"det(A.T) = {det_AT:.4f}")

--- 
# 5. Banco de Ejercicios Prácticos (30+)
Ahora te toca a ti. Resuelve estos ejercicios para consolidar tu conocimiento.

### Parte A: Cálculo e Invertibilidad

**A1 (🟢 Fácil):** Calcula el determinante de $A = \begin{pmatrix} 5 & 2 \\ 3 & 1 \end{pmatrix}$ y determina si es invertible.

**A2 (🟢 Fácil):** ¿Es invertible la matriz $B = \begin{pmatrix} -8 & 4 \\ -2 & 1 \end{pmatrix}$? Justifica tu respuesta calculando su determinante.

**A3 (🟢 Fácil):** Calcula el determinante de la matriz identidad 3x3. ¿Tiene sentido el resultado?

**A4 (🟡 Medio):** Genera una matriz singular 4x4 usando `create_special_matrices`. Verifica que su determinante es (casi) cero.

**A5 (🟡 Medio):** Dada la matriz $A = \begin{pmatrix} 1 & 0 & 2 \\ 2 & -1 & 3 \\ 4 & 1 & 8 \end{pmatrix}$, calcula su determinante.

**A6 (🔴 Reto):** Demuestra con un ejemplo 2x2 que, en general, $\det(A + B) \neq \det(A) + \det(B)$.

### Parte B: Interpretación Geométrica

**B1 (🟢 Fácil):** Dada la matriz de escalado $A = \begin{pmatrix} 3 & 0 \\ 0 & 2 \end{pmatrix}$, ¿por cuánto multiplica el área de cualquier figura? ¿Preserva o invierte la orientación? Responde calculando el determinante.

**B2 (🟢 Fácil):** ¿Qué le hace al área una matriz de rotación? Genera una matriz ortogonal 2x2 y calcula su determinante para confirmarlo.

**B3 (🟡 Medio):** Construye una matriz 2x2 que invierta la orientación y triplique el área. Visualiza su efecto con `plot_transformation_area`.

**B4 (🟡 Medio):** La matriz de cizalla (shear) $S = \begin{pmatrix} 1 & k \\ 0 & 1 \end{pmatrix}$ no parece que debiera cambiar el área. Calcula su determinante para verificar esta intuición.

**B5 (🔴 Reto):** Si aplicas una transformación $A$ y luego otra $B$ a una figura, el área se escala por $\det(B)$ y luego por $\det(A)$. El factor de escala total es $\det(A) \cdot \det(B)$. La transformación combinada es $A \cdot B$. Verifica numéricamente que $\det(A \cdot B) = \det(A) \cdot \det(B)$ para dos matrices 2x2 aleatorias.

### Parte C: Aplicaciones y Retos con Datasets

**C1 (🟡 Medio):** Toma los primeros 30 puntos de `datos_estudiantes`. Calcula la matriz de covarianza y su determinante. Luego, toma los últimos 30 puntos y haz lo mismo. ¿Qué subgrupo de estudiantes está más "disperso"?

**C2 (🔴 Reto):** Crea un nuevo dataset de estudiantes `datos_correlacionados` donde la calificación sea casi perfectamente `calificacion = 10 * horas + ruido_pequeño` (usa `rng.normal(scale=0.1)` para el ruido). Calcula su matriz de covarianza y su determinante. Compara este determinante con el del `datos_estudiantes` original y explica por qué es mucho más pequeño.

**C3 (🔴 Reto Conceptual):** Imagina que tienes un dataset de 3 features y calculas su matriz de covarianza 3x3. Si el determinante de esta matriz es cero, ¿qué te dice esto sobre tus features? ¿Qué problema práctico implica?

---

## ✅ Mini-Quiz de Autoevaluación

1. Si `det(A) = -4`, ¿qué le hace la transformación A al área de una figura en 2D y a su orientación?
2. Verdadero o Falso: Si el determinante de una matriz es diferente de cero, sus columnas son linealmente independientes.
3. En el contexto de datos, un determinante de la matriz de covarianza cercano a cero sugiere que los datos presentan alta... (completa la palabra).
4. ¿Se puede calcular el determinante de una matriz 3x4? ¿Por qué?
5. ¿Cuál es el determinante de cualquier matriz de reflexión (como un espejo) en 2D?

## 🚀 Próximos Pasos

¡Felicidades! Has conectado el determinante con una medida estadística clave y una profunda intuición geométrica.

- En el notebook final de esta sección, **`1.1.3.3_Eigenvectores_y_Eigenvalores`**, usaremos el determinante para encontrar la 'esencia' de una matriz: aquellos vectores especiales que solo son escalados por la transformación. Estos nos revelarán los ejes fundamentales de acción de una matriz, un concepto que es la base del Análisis de Componentes Principales (PCA).